In [ ]:
!pip3 install shapely==1.7.1

In [ ]:
!pip install google-cloud-aiplatform
!pip install openai
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.0/321.0 kB 44.5 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.0 MB/s eta 0:00:00


In [ ]:
from langchain.llms import VertexAI
from langchain import PromptTemplate, LLMChain

In [ ]:
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/ucr-ursa-major-maverics-lab-5986748a3acf.json"


In [ ]:
import pandas as pd
dftest=pd.read_csv("/content/extractive_test.csv")
dftrain=pd.read_csv("/content/extractive_train.csv")

In [ ]:
dftrain.head()

,Article ID,Original Article,Highlight,Generated Summary,R1-R,R1-P,R1-F,R2-R,R2-P,R2-F,RL-R,RL-P,RL-F
0,469c6ac05092ca5997728c9dfc19f9ab6b936e40,"marseille , france -lrb- cnn -rrb- the french ...",marseille prosecutor says `` so far no videos...,"marseille , france -lrb- cnn -rrb- the french ...",0.3617,0.3696,0.3656,0.2037,0.1964,0.2000,0.3404,0.3478,0.3440
1,f001ec5c4704938247d27a44948eebb37ae98d01,-lrb- cnn -rrb- the palestinian authority offi...,membership gives the icc jurisdiction over al...,-lrb- cnn -rrb- the palestinian authority offi...,0.3226,0.2222,0.2632,0.1765,0.1017,0.1290,0.3226,0.2222,0.2469
2,e2706dce6cf26bc61b082438188fdb6e130d9e40,-lrb- cnn -rrb- governments around the world a...,amnesty 's annual death penalty report catalo...,-lrb- cnn -rrb- governments around the world a...,0.4783,0.3333,0.3929,0.2321,0.1548,0.1857,0.4565,0.3182,0.3532
3,c222979bd1cfbc7d3ff821e9c738e3dbd29b14f4,"-lrb- cnn -rrb- on may 28 , 2014 , some 7,000 ...",amnesty international releases its annual rev...,"in the patently unfair , open air trial that f...",0.1765,0.1364,0.1538,0.0000,0.0000,0.0000,0.1765,0.1364,0.1490
4,c27cf1b136cc270023de959e7ab24638021bc43f,-lrb- cnn -rrb- a duke student has admitted to...,student is no longer on duke university campu...,-lrb- cnn -rrb- a duke student has admitted to...,0.7308,0.3958,0.5135,0.2414,0.1186,0.1591,0.6538,0.3542,0.3953


In [ ]:
P=['0408464fb6eb2702626d2ca1cbb7c28e76ddd255', '18f7505fa80eaa981a2f52fbb96fcea5908cd4ee', '97163bb36cb49730a634a0cbaa01108c2fd89e01', 'cd421645a56af17128c17a41ec2c9074bce84ddc', '4fde6cc846d0709e25a9cccec8d197ba26c15c82', '5f5a8814d8ec3c16299782db7c724cb771ea5adb', '56516cebdf30bd22dc8353549b70b4bb33993753', '446fdf91f59be259167ae59b0a3de1a664891751', '3d58a02a6578e08c2bf7494263ef9a083620e1f6', 'c077e15f710484d2cb3ffe26cd9b331353cb74e2', 'fd7a007f17a13ee329cba78b298bdd77d4d33d42', 'ba2ec653b7d0c46759f39aef6dac850c9229716f', '4cfe02fe65eedf9364dd2b9633dca27349ec02aa', '58b0d34958a722a453fe24deed240e08e7be8ced', '6abc848961744dde2da8a4b8e5337f34d32d45ed', '62545d81f24bc768afd19875d9e62e38a6ddb97d', 'efdb6043cf2ac2663137f8cb8696f0735ee58b86', 'bde8e22f3e1c3af2ca0f09f734f33d67140ef1aa', '9eccb748cdaefd4c472101db76f347914f850c1a', '685b0b902225d5b807a721e294f5ae16febf1f0e', 'b4c8c6a6ad6257b5d8dfd82b80cd3b0fbdd342e3', 'e79e40746dbc24643a3a3f044d2f49689ff82e38', '539d895fd424cbda3e4985c6aee36e272401b359', 'db61d01f82dc63596f7e2fa93b4ab4ae278daf1f', 'c331159c09eee2e7a27b90d973500e7b3db04d94', '1f4aaf00cad1c6272240bd109fd34201b92a0937', '436772093dd75272d1d867c80e6b142329fcfa3e', '6de323acb06b2590f5aba38ae450865ea4e793b1', 'ebccdbf5779c2d3fa7180747e333dde756aa62fb', '2bc2c4566f364d664331809744ff9a40841d9617', '975132a0649fbebbdb6cdd32534b165eece0053a', '3612c620d3fa1eafc99c0c154ad5caf7481727f4', '31edccbcedf3d7e2819818f109a35ecdfcd65970', 'b8d8f6a6518138c8b1e0e845f46e62e2b7fe3fd8', 'de45d9d92d7ee486a71ba13f4cc52470b9df5c01', 'be8918aac4ce45cfefddaa245c26d9314d1194d4', '3415b0e398cd14d27f386b28a61f4d585b8eaa11', '7623d447bab49c8e99dfc99289b2281a1fed8d4f', '45071cfaf26fa14f1a8a9276393401f54e0f93ca', '837970f6b544e927c6fe2261a81f74f8313f2bdf', '47609113b2709b46ccafb3782980cd3da7ab7d2d', 'c446f86eea5f7d48f5cae27df669b80aff8de043', 'a84811a80358e86f9a62669fa69521a0ac12130a', 'bbdc0e58a203b016166f896936d9862ae9fd11be', '6c5ef5f577b97c98b8580476690883176584569e', '6a778af88ba885ef38fd88987afa1b6167e5c494', '2094d594b67b61d842fdc5ef4891b721e36171e7', '6af327c6e93d12422c1b605ff7581a813cdb1691', 'c94b92e2e7f31e2ed580137a02a5c60fa6537458', '540fc636957b4d049d6b4f07ae72c1b3be6ec020', 'c8fcf4c23a15895a2c7272ec04c5a92b06575d5e', '3a407cf2ec50f113c6300b7a39936c229d1b29f4', '1d17d402757c7da68c499b95368f477ad5d66513', 'a382fc98c0652dea0c2275a4110b744476b30ea3', '949378fdf5114619911a5e0fe0182d45f1c27c17', '9dad1056763b4580aed22d85670a801c8c3f352c', '644da4a1c3baef155cf8c4675008540a8b777988', '35d5a717be7c0144d24c34233e5321e871dc614e', '8000bfa6f05e428b2c7cc552d0df4c36283108fd', '7be8bc293e448903d2b23f4cea091b0407848897', 'bfcee0d2eed31196bb00e1f52606a3f168eee17c', '2b4eb5a283f51b54e1b0318936ae9bcfa70ce243', 'd75247e02a692fc343bc9826662870011eaed521', '4b637c62b6ac747871a3051a97f29dd282633360', '270fdae145aa5ae3ae73adcf44be72cec339dbbe', '60c3114cd31fced2982a5e912f2224220d6c5254', '32259d1946a7e376f3915aea7f7b4a4e14cbe02f', '80a28afe6ded1ad0ddd6f2ba0ac9b66436e91739', 'c3881866bcacbb55bd959b15a197bae3eb8b3384', 'f5ff04170c39e33afdf58bb732dd03265662c4c0', 'cb7a72d78d5f34b9300f8486e40e57ffcf9d98d5', 'f1beee2435f59b05e0da484c7a1c167709412d6d', 'e1502ae8cb04cbdceee368551cfd65bcf6a16f07', 'e6e0a4b568f6214c18c38cfcac5a10ecfb0dbf6e', '623ad83ebb32752afaa1fb4b134ab697eae8b798', '0e6dd3f40daaff5c14f0a34ba75e9f05c40a55c8', '0d1d6284061a99cb2478ecb4b490e4605698ec3e', 'de24115848174d0534512cd251b1d53cd343f23e', '574767c29dfee14ea227639f45387c07437a55e5', 'a8a2ace82e62b4020b5a241060d4a548f7ee3a7f', 'bef9d7ac62dac0fb9a200d2acae6d333388ec7f1', '6426eaaa8fe11407ed2290d9c0ad2bc8e1a723d5', 'bb8a7a6bc30f5d10bfd747daa4887134a780d66d', 'f7dd3331408529408be2b706392e0b4a02a6a4cc', '84a933b7edc70c7f5e6ed6558c7b57c100abc114', '9f4ab634ce7925b5527345ba0efc583b206c47a2', 'f800996f2cca42b205a7cfd37805fb77d577254b', 'd6f928d950f7d97c9fc3a5f575b9eaa325a7dd4d', '8f324e3064f6cdc9c378dfcf4545ed2c124e7103', '32069f98ff75ce1a65beeffcf84cd80f92bae5dc', '7f13d9dff7707609a042a5c3e99671c98de8c135', '4dcf206f63daa3ffdb4c77b2441e7aed78d2c8c5', '864240ad271c7da6c74823abf2abfb2b98f427cb', '6299e17b582fffbd6732d237dcc06c97c5d935ff', 'c3cb61c858ec4a178760b619430ddff7e03eb0c8', '0a8fd2888f20c9e705696d8ef847a32df47900e6', 'b68be4e8fe382de849954933357e4a32409c20ff', '34f1b80e4e3fbd60dd942b3e7e52a7f8c61f06ef', '52343da9a3ef719280b4b6946e6c500e819552d0', '2f6106809de41d59f422d527900a6b344174c5b2', '69406e34836ea1f7680d4298e83fd6d83a6c7997', 'ef2d42928cc3b1762d0bbb6037db308db53fc864', '31809d739569d22b663fc1cdbfb1fb4206ade734', '98d980ba9bc5479f9c340a18a61941d5ccb52774', 'dbba46c94632ad510072e1b52c4f427c4dadca60', '65676acfb425e2a29cd7fca216174635ea408382', '16b9db5d22a87f739c918886c6b3d7acdfcfd58a', '30f2056f43c0a58a261c4feddefd131d498199cb', '5231a93ff48e3dcdc22ea9df981237c26a474714', '29503dc74ccb039c4c1562844299116a545dea39', 'd34233997a6d71174a09a8d8001ec64468a2f640', 'a37aa9e7b6985f710c3ad79cd0b53619b5e0bf5a', '707c09009396d151d2d4f819925f78e446151f39', '33ca9fba99197c82369e99aa3d234783e719227c', 'faf1d451489b43676734ea706926885bbbedfce3', '0c5ec67f7b1d7a2fdb0a3f3ffb8263d8d9ddf872', 'd9e8e6f9c0255bf26881a2eabba8f5cbe733c55f', '9625484feb731f45fd8059112238d94f141d413c', 'ca3b2dc93f170f06e3a60fe7cd3243d204cc4b75', '2103c883437dbd9f0c17885f60190dabb4ab56dc', '30944082061b9e844391dcbc98775f22cb3ef791', '202c35ec48be21fc196139dd538db271c0f65e4a', '280d5930fb82ad2958fd1dcf452c6a4f1b5ad39f', '71efc54d9a237c40c469965cc24841f9f9a7b019', '3291399360e20650ea565c27371a72c7b448082e', 'e67604ecb75aa21391e1d15a577d16b8db990de5', 'e2a44a3e3edd44c394ad6ff7207980f7f1e9b4b3', '7661803c32a1511bf302a8a5abdc3c462f080fb2', 'de84417cc1ffcb6ecabfab7600c232eb9fd11915', '8a7466a5c8516b2087aea24647cc07b508697ab6', '20a6b5894c4cd75e32f6f4946467854aeb539337', '7113eca13e8f3dd7760e609fdc97660de1806d2a', '61254109741e1db17ba31582d1c783d282e6bbf6', '78b863c5bce9b8fc0d743bc396b0308a9eb17a77', '92151f31c3478c5e0e4352a0f7323c3acfc297e7', 'cabca761a3814275d3d1b2b6c7cfc1f9cd3810b7', '878c4af5ebfb7a1831af90ad0d6e7bdcc4a3d4b8', '1dd00c89d71a5611797bb34da8a6eada8f058405', '711e00313310a956641a47791fdf8299ee06b2c2', '01b0585316ca58505df1d4ace2d888d0e50e7600', 'c11e656ad31a4f14b0d276290a757a0da6a73ba4', '4b132f90a0f4369df9b7ecba759c5aa068dd12be', 'd3e9f0aca4e608fc2a3b85115b96cb0c453191af', 'c18fe1d648b40aa763644c37a9a2a562a340e172', '9ad4662fc2d978672ae4b5eea4e4806ae6a7d076', 'db9f358d0e08da7cd0276453f8bcdc114ff06fdb', 'b19ab7fa421bcbb921c5a20718db5918091bb15d', '9268b9d826db3d8acf31a03bdc3e30cd75cc93de', 'be5d8524bf21d485e203425d764ed60a3bf71b8d', 'f97b9b19e5a90dce0fc0b098f2a7f70ec36445fb', 'e84c74b45a589a6faefaa881016505fd282a46ca', 'd495bb584d5fda76f9b90beca528469f5bcfc5b3', 'f7f9156f26bbf12e63523bdd2f607a2f7a7f9f0c', '7e91926e47bdaa4054e3a69784bee8ee27d0dd4b', '2434b0ec4419fd48c0d15e543030555129398ec6', 'f0dba7fa5d45ffb29070afda0c4b230c8f318a51', '82461522ee90f8f5be2ec226e2016ad40588866d', 'a3830cf0f4311a3f816a710d61d7e14ac9ffa9fe', 'bc9d5ab2c343edd6522ee4ebe63c5ca4adcebb1b', '9638b449931f0f4a21167997d37d5dd43ee2430f', 'e8b93366b9c9c5bf4409dc1f8064122063ccd4ca', '3a8e2afdba04d7356526c94232f17eef9a5e4ddd', '7ae16f8ac4a86a6de4669e130e068d9f7e492ddf', '1b3bdc7fa131c4c82e8e996c936abe0da840e2ef', '5d1577700e760d853376d15ad75d8972119d6ddb', 'd6bbbfa9b5c58e99d21d549a9fb5b512b342e301', 'e0a415bf04e1929507bb8626cdf6379169264838', '5c3e16c415b463269527cafd220170e1303048f8', '09085a0b380bcbc205c7a3902f4d5492da2eacda', '223ecfeb056056991eb877fbd63ce66ca616d372', 'f708809379316d68524802c6ca052985bbe248cd', 'c38207a8ba7332bf8fc449770b2554510fa1482a', '3bdec3398b4dd68453d97ab7718086655843e899', 'a7827e12988bd89e4f1079705079a135beefc1f2', '34c33427589efd00f343e49e2e5109d444d560f5', '198f9a4a72b169e04ec003cb4717f46dc0d3191f', '9ef9bcf7b102537cff2774dd7be2c6eefd5b80fd', '7e4d197d5fe8272c0471199cf9987f3786a80c2b', '1fda6d5647b826db2ab920c945dd7944dbdfbde9', '2805d03b2ec2d8a1e5343f71d70912db2a58736e', 'ddcbe22893aa887ff799cf3e40bbe2a536965f91', 'b6d1933d8d4a8c1b6b76d44ec3ed69d86268a7b3', 'ad58f533f9eaf52080929e32c5dd5eaf2bafba0e', 'f2a9f3c80876424da8cee335cb0e0467caa5f3ca', '5ba4a7a9733a497625971981764184cb75618b81', 'a659d867e198efa85e02b90d5e78d2f6a6e60fb7', '36c34f4ffa8815711beea7d2fda86eea047b8802', 'a4b00d5247eef69b5d51cab1f1883da314211366', 'cca91bf7af4db82293421776f5299457dce34909', 'e248a8e2416dddfa8899f0781b967429232063b3', 'e266b3daac6d9e72e637e8f83db97621c5beafd0', '9f2dbe8953dd5b1d2b0f2f748c51bfccbe72906e', 'a2b66e4376258cd41604d654a395f7540d2e8514', '2edc424a6d80e2e10a46b1e8b52f14c481cc1daa', '0cdf69eb6b6860142724e940cf9f75d93907227f', '34ad340fa41e205b95b37b00e8ccbc38afb3e68f', '117444537cbffcb807dc04565f39dca46f7d2cef', 'ecb7f439cc992286b8933b117bb6c498bd133bcd', 'b6ee1b3428dda9063461deefc0086c2d66279d0f', '42d88bafeb5f838a0be02c3eb97fe12f5df3d92f', '4c976ed4cb92068d476b1c17753715948e39c62f', 'a5fffe339cda4c8e002d992ab93888559107f3ab', 'd6fc70d27119a3705b96ace6b20b9fd2f3cfcbca', '89fed7685538cd358e611932653eea931d7b057f', '5effc218f677590dbc9037e196549b5341242933', 'b8a47e9569e9d0919638f5bed6f13e565f84e855', '35c590581dcd2a417101f2ef80f63f69cdfe4030', 'f99b61fb576e893ffbd2e6d6f494b9dd0fbf8941', '5dd71e881a79e27363e2f73c3afa80b8cc3e5a64', '64b3b703816b40851bd4b7c0255df5834e136611', 'cf5e6722047569c81b635be96ef0b07cb04eed73', 'ffd4fd3ed5e3d498d6135fc93ae6c9170d10611c', '159ab1d0f729f77e86a6a77bf43d0291d069c66c', 'e0a3acedc34d92569ce5f6032d504c35fbf30720', '0e5130d172cdaaa2afcb51585f4aa0580b049431', 'a776fa2caa0fa9cac0ce5bf71d2bf44eb7a7346a', '1027d076dbb423b90447f6f3f8315610aefd682c', '0322c7a3366d45134e8ba78c7f369d2f2cd14176', '0db1a9abd0524d51270531e3a80d36afaa7550fe', '0d29ae06dd015e141646c3b03671901c1b8465c4', 'e2ef75518e69947f79acb9202516689b6c869e92', '16bf321c82f6ffe7f20479a12a9e432685f2d91b', 'c37431a19d8433f20447858d1c810b7b77804473', '77685c1f281a3e7472b51f6619d7d54c6bc0c717', '6d76062fb07e8a37609bbd527934a9219929560b', '083b6c3239f969ae12491f14fc09b1f17d396983', '1afd2df29d4e642f20130a231d27004216667fe1', '8749719d4fb9a14d833d39cde0b6e07985bee823', '43959f19ee64571fead0661415d4e13c54d71ee1']


In [ ]:
len(P)

229

In [ ]:
template = """Write a concise summary for the following: "{a}." """
prompt = PromptTemplate(template=template, input_variables=["a"])

# Assuming you have the following modified get_summary function
def get_summary(a, retries):
    llm = VertexAI(
        project='ucr-ursa-major-maverics-lab',
        model_name="text-bison@001",
        temperature=0.2,
        max_output_tokens=1024,
        top_p=0.8,
        top_k=40
    )
    if retries < 0:
        return None
    llm_chain = LLMChain(prompt=prompt, llm=llm)
    b = llm_chain.run(a)
    if 'article' in b or b == '' or b is None:
        return get_summary(a, retries-1)
    else:
        return b

# List to store problematic article IDs
problematic_files = []

# Function to generate summary for a single article
def generate_summary_for_one(article_id, article_content):
    summary = get_summary(article_content, 25)
    if summary is None:
        problematic_files.append(article_id)
        return None
    return summary

# Function to generate summaries for specific article IDs and save to CSV
def generate_csv_for_specific_ids(article_ids, df):
    # Create an empty DataFrame to store results
    results_df = pd.DataFrame(columns=['Article ID', 'LLM Summary'])
# Iterate over the specific article IDs
    for index, article_id in enumerate(article_ids):
        article_content = df[df['Article ID'] == article_id]['Original Article'].iloc[0]
        summary = generate_summary_for_one(article_id, article_content)
        if summary is not None:
            results_df = results_df.append({'Article ID': article_id, 'LLM Summary': summary}, ignore_index=True)
        print(f"Finished processing article {index + 1} of {len(article_ids)}")

    # Save to CSV
    results_df.to_csv('llm_summaries.csv', index=False)


# Generate summaries for the specific article IDs
generate_csv_for_specific_ids(P, dftest)

# Print problematic article IDs
print("Problematic Article IDs:", problematic_files)


Finished processing article 1 of 229
Finished processing article 2 of 229
Finished processing article 3 of 229
Finished processing article 4 of 229
Finished processing article 5 of 229
Finished processing article 6 of 229
Finished processing article 7 of 229
Finished processing article 8 of 229
Finished processing article 9 of 229
Finished processing article 10 of 229
Finished processing article 11 of 229
Finished processing article 12 of 229
Finished processing article 13 of 229
Finished processing article 14 of 229
Finished processing article 15 of 229
Finished processing article 16 of 229
Finished processing article 17 of 229
Finished processing article 18 of 229
Finished processing article 19 of 229
Finished processing article 20 of 229
Finished processing article 21 of 229
Finished processing article 22 of 229
Finished processing article 23 of 229
Finished processing article 24 of 229
Finished processing article 25 of 229


<ipython-input-13-cd5cea4e0366>:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'Article ID': article_id, 'LLM Summary': summary}, ignore_index=True)


Finished processing article 26 of 229
Finished processing article 27 of 229
Finished processing article 28 of 229
Finished processing article 29 of 229
Finished processing article 30 of 229
Finished processing article 31 of 229
Finished processing article 32 of 229
Finished processing article 33 of 229
Finished processing article 34 of 229
Finished processing article 35 of 229
Finished processing article 36 of 229
Finished processing article 37 of 229
Finished processing article 38 of 229
Finished processing article 39 of 229
Finished processing article 40 of 229
Finished processing article 41 of 229


<ipython-input-13-cd5cea4e0366>:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'Article ID': article_id, 'LLM Summary': summary}, ignore_index=True)


Finished processing article 42 of 229
Finished processing article 43 of 229
Finished processing article 44 of 229


...

Finished processing article 118 of 229
Finished processing article 119 of 229
Finished processing article 120 of 229


<ipython-input-13-cd5cea4e0366>:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'Article ID': article_id, 'LLM Summary': summary}, ignore_index=True)


Finished processing article 121 of 229
Finished processing article 122 of 229
Finished processing article 123 of 229
Finished processing article 124 of 229
Finished processing article 125 of 229
Finished processing article 126 of 229


<ipython-input-13-cd5cea4e0366>:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'Article ID': article_id, 'LLM Summary': summary}, ignore_index=True)


Finished processing article 127 of 229
Finished processing article 128 of 229
Finished processing article 129 of 229
Finished processing article 130 of 229
Finished processing article 131 of 229
Finished processing article 132 of 229
Finished processing article 133 of 229
Finished processing article 134 of 229


In [ ]:
template ="""Write a concise summary for the following: "{a}."
    """
prompt = PromptTemplate(template=template, input_variables=["a"])

import pandas as pd

# Assuming you have the following modified get_summary function
def get_summary(a, retries):
    llm = VertexAI(
        project='ucr-ursa-major-maverics-lab',
        model_name="text-bison@001",
        temperature=0.2,
        max_output_tokens=1024,
        top_p=0.8,
        top_k=40
    )
    if retries < 0:
        return None
    llm_chain = LLMChain(prompt=prompt, llm=llm)
    b = llm_chain.run(a)
    if 'article' in b or b == '' or b is None:
        return get_summary(a, retries-1)
    else:
        return b

import pandas as pd

# List to store problematic article IDs
problematic_files = []

# Function to generate summary for a single article
def generate_summary_for_one(article_id, article_content):
    summary = get_summary(article_content, 10)
    if summary is None:
        problematic_files.append(article_id)
        return None
    return summary

# Function to generate summaries for all articles and save to CSV
# ... [rest of the code remains unchanged]

# Function to generate summaries for all articles and save to CSV
def generate_csv_for_two(df):
    # Create an empty DataFrame to store results
    results_df = pd.DataFrame(columns=['Article ID', 'LLM Summary'])

    # Iterate over the first two rows of the input DataFrame
    for index in range(5):  # This will loop over indices 0 and 1
        article_id = df.loc[index, 'Article ID']  # Assuming you have a column named 'Article_ID'
        article_content = df.loc[index, 'Original Article']
        summary = generate_summary_for_one(article_id, article_content)
        if summary is not None:
            results_df = results_df.append({'Article ID': article_id, 'LLM Summary': summary}, ignore_index=True)

    # Save to CSV
    results_df.to_csv('llm_summaries.csv', index=False)

# Generate summaries for the first two articles
generate_csv_for_two(df)

# Print problematic article IDs
print("Problematic Article IDs:", problematic_files)




<ipython-input-34-b2602b03ece3>:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'Article ID': article_id, 'LLM Summary': summary}, ignore_index=True)
<ipython-input-34-b2602b03ece3>:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'Article ID': article_id, 'LLM Summary': summary}, ignore_index=True)
<ipython-input-34-b2602b03ece3>:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'Article ID': article_id, 'LLM Summary': summary}, ignore_index=True)
<ipython-input-34-b2602b03ece3>:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df

Problematic Article IDs: ['0408464fb6eb2702626d2ca1cbb7c28e76ddd255']


In [ ]:
# Import necessary libraries
import pandas as pd
from rouge import Rouge

# Define the calculate_rouge function
def calculate_rouge(hypothesis, reference):
    rouge = Rouge()
    scores = rouge.get_scores(hypothesis, reference, avg=True)

    # Extract scores and format them into the desired dictionary format
    formatted_scores = {
        'llmR1-R': scores['rouge-1']['r'],
        'llmR1-P': scores['rouge-1']['p'],
        'llmR1-F': scores['rouge-1']['f'],
        'llmR2-R': scores['rouge-2']['r'],
        'llmR2-P': scores['rouge-2']['p'],
        'llmR2-F': scores['rouge-2']['f'],
        'llmRL-R': scores['rouge-l']['r'],
        'llmRL-P': scores['rouge-l']['p'],
        'llmRL-F': scores['rouge-l']['f']
    }
    return formatted_scores

# Read the llm_summaries.csv file into a DataFrame
results_df = pd.read_csv("llm_summaries.csv")

# Assuming df contains the reference summaries and is already loaded
# Calculate ROUGE scores and split them into separate columns
rouge_scores = results_df.apply(lambda row: calculate_rouge(row['LLM Summary'], df[df['Article ID'] == row['Article ID']]['Highlight'].values[0]), axis=1, result_type='expand')

# Concatenate the original DataFrame with the new ROUGE score columns
results_df = pd.concat([results_df, rouge_scores], axis=1)

# Set the display options to show the full content
pd.set_option('display.max_columns', None)

# Print the DataFrame with the new ROUGE score columns
results_df


,Article ID,LLM Summary,llmR1-R,llmR1-P,llmR1-F,llmR2-R,llmR2-P,llmR2-F,llmRL-R,llmRL-P,llmRL-F
0,517a2943ee6fab9e6a81bd5cd259e5801682d190,"Jack Grealish is a 19-year-old academy graduate who has been given his first Premier League start by Aston Villa manager Tim Sherwood. He has impressed in his two games so far, playing a part in Christian Benteke's equaliser against Queens Park Rangers and setting up Fabian Delph's winner against Liverpool. Sherwood believes that Grealish has a big future at the club and is a player who relishes the big stage.",0.137931,0.072727,0.095238,0.000000,0.000000,0.000000,0.137931,0.072727,0.095238
1,e66247c8e4f0f7907e9c040bf7064782dfcb3323,"A Wisconsin pizza company is being mistaken for another pizzeria in Indiana that made headlines after its owners said they would refuse to cater a same-sex wedding.\nThe owners of Memories Gourmet Pizza in Nichols, Wisconsin, say they have been getting angry calls and messages on social media from people who think they are the Memories Pizza in Walkerton, Indiana.\nThe owners of the Wisconsin pizzeria say they have nothing to do with the Indiana pizzeria and do not agree with their views.\nThey are worried that the mix-up could damage their business.",0.400000,0.215385,0.280000,0.075000,0.034483,0.047244,0.371429,0.200000,0.260000
2,907fe50dbe4dc0fb84b0683fab578806536dba28,"Shay Given will start for Aston Villa against Liverpool in Sunday's FA Cup semi-final at Wembley. The 38-year-old has played in all four of Villa's FA Cup matches this season and has kept two clean-sheets. Villa manager Tim Sherwood confirmed Given will start on Sunday, a day before the Irishman turns 39. Sherwood believes Villa can upset favourites Liverpool.",0.527778,0.395833,0.452381,0.225000,0.163636,0.189474,0.527778,0.395833,0.452381
3,41a572feb1c9f4651b9b1e78f1482b92e16a63b1,Amir Khan believes Kell Brook has called him out so that he can have the ` one big payday ' which has so far eluded him during his 10-and-a-half year career .\nBrook is desperate to fight Khan but his fellow Brit rejected the chance to face him at Wembley in June .\nKhan claims that Brook is not even a big name in the UK and is not able to face the world 's biggest names .,0.317460,0.370370,0.341880,0.083333,0.098592,0.090323,0.301587,0.351852,0.324786


In [ ]:
!pip install rouge



ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_i

In [ ]:
template ="""Summarize this: {a}."
    """
prompt = PromptTemplate(template=template, input_variables=["a"])

import pandas as pd

# Assuming you have the following modified get_summary function
def get_summary(a, retries):
    llm = VertexAI(
        project='ucr-ursa-major-maverics-lab',
        model_name="text-bison@001",
        temperature=0.2,
        max_output_tokens=1024,
        top_p=0.8,
        top_k=40
    )
    if retries < 0:
        return None
    llm_chain = LLMChain(prompt=prompt, llm=llm)
    b = llm_chain.run(a)
    if 'article' in b or b == '' or b is None:
        return get_summary(a, retries-1)
    else:
        return b

import pandas as pd

# List to store problematic article IDs
problematic_files = []

# Function to generate summary for a single article
def generate_summary_for_one(article_id, article_content):
    summary = get_summary(article_content, 10)
    if summary is None:
        problematic_files.append(article_id)
        return None
    return summary

# Function to generate summaries for all articles and save to CSV
# ... [rest of the code remains unchanged]

# Function to generate summaries for all articles and save to CSV
def generate_csv_for_two(df):
    # Create an empty DataFrame to store results
    results_df = pd.DataFrame(columns=['Article ID', 'LLM Summary'])

    # Iterate over the first two rows of the input DataFrame
    for index in range(5):  # This will loop over indices 0 and 1
        article_id = df.loc[index, 'Article ID']  # Assuming you have a column named 'Article_ID'
        article_content = df.loc[index, 'Original Article']
        summary = generate_summary_for_one(article_id, article_content)
        if summary is not None:
            results_df = results_df.append({'Article ID': article_id, 'LLM Summary': summary}, ignore_index=True)

    # Save to CSV
    results_df.to_csv('llm_summaries2.csv', index=False)

# Generate summaries for the first two articles
generate_csv_for_two(df)

# Print problematic article IDs
print("Problematic Article IDs:", problematic_files)




<ipython-input-32-c14482cdfc3b>:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'Article ID': article_id, 'LLM Summary': summary}, ignore_index=True)
<ipython-input-32-c14482cdfc3b>:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'Article ID': article_id, 'LLM Summary': summary}, ignore_index=True)
<ipython-input-32-c14482cdfc3b>:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'Article ID': article_id, 'LLM Summary': summary}, ignore_index=True)
<ipython-input-32-c14482cdfc3b>:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df

Problematic Article IDs: ['0408464fb6eb2702626d2ca1cbb7c28e76ddd255']


In [ ]:
from rouge import Rouge

def calculate_rouge(hypothesis, reference):
    rouge = Rouge()
    scores = rouge.get_scores(hypothesis, reference, avg=True)
    return scores

results_df=pd.read_csv("llm_summaries2.csv")
# Add a new column to your results_df for ROUGE scores
results_df['ROUGE Score'] = results_df.apply(lambda row: calculate_rouge(row['LLM Summary'], df[df['Article ID'] == row['Article ID']]['Highlight'].values[0]), axis=1)

# Print the ROUGE scores
# Set the display options to show the full content
pd.set_option('display.max_colwidth', None)

# Print the ROUGE scores
print(results_df[['Article ID', 'ROUGE Score']])

                                 Article ID  \
0  517a2943ee6fab9e6a81bd5cd259e5801682d190   
1  e66247c8e4f0f7907e9c040bf7064782dfcb3323   
2  907fe50dbe4dc0fb84b0683fab578806536dba28   
3  41a572feb1c9f4651b9b1e78f1482b92e16a63b1   

                                                                                                                                                                                                                                                             ROUGE Score  
0                                        {'rouge-1': {'r': 0.06896551724137931, 'p': 0.05128205128205128, 'f': 0.0588235245198966}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.06896551724137931, 'p': 0.05128205128205128, 'f': 0.0588235245198966}}  
1  {'rouge-1': {'r': 0.22857142857142856, 'p': 0.11940298507462686, 'f': 0.15686274059015776}, 'rouge-2': {'r': 0.025, 'p': 0.012345679012345678, 'f': 0.016528921193908704}, 'rouge-l': {'r': 0.22857142857142856, 'p': 0.11940298507

In [ ]:
from langchain.llms import VertexAI
from langchain import PromptTemplate, LLMChain

import pandas as pd

import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "./ucr-ursa-major-maverics-lab-5986748a3acf.json"
df=pd.read_csv("./extractive_test.csv")


template = """Summarize this: {a}."""
prompt = PromptTemplate(template=template, input_variables=["a"])


# Assuming you have the following modified get_summary function
def get_summary(a, retries):
    llm = VertexAI(
        project='ucr-ursa-major-maverics-lab',
        model_name="text-bison@001",
        temperature=0.2,
        max_output_tokens=1024,
        top_p=0.8,
        top_k=40
    )
    if retries < 0:
        return None
    llm_chain = LLMChain(prompt=prompt, llm=llm)
    b = llm_chain.run(a)
    if 'article' in b or b == '' or b is None:
        return get_summary(a, retries-1)
    else:
        return b

import pandas as pd

# List to store problematic article IDs
problematic_files = []

# Function to generate summary for a single article
def generate_summary_for_one(article_id, article_content):
    summary = get_summary(article_content, 10)
    if summary is None:
        problematic_files.append(article_id)
        return None
    return summary

# Function to generate summaries for all articles and save to CSV
def generate_csv_for_all(df):
    # Create an empty DataFrame to store results
    results_df = pd.DataFrame(columns=['Article ID', 'LLM Summary'])

    for index, row in df.iterrows():
       article_id = row['Article ID']  # Assuming you have a column named 'Article_ID'
       article_content = row['Original Article']
       summary = generate_summary_for_one(article_id, article_content)
       if summary is not None:
            results_df = results_df.append({'Article ID': article_id, 'LLM Summary': summary}, ignore_index=True)

    # Save to CSV
    results_df.to_csv('llm_summaries.csv', index=False)

# Generate summaries for all articles
generate_csv_for_all(df)

# Print problematic article IDs
print("Problematic Article IDs:", problematic_files)


<ipython-input-7-8fa859f51091>:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'Article ID': article_id, 'LLM Summary': summary}, ignore_index=True)
<ipython-input-7-8fa859f51091>:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'Article ID': article_id, 'LLM Summary': summary}, ignore_index=True)
<ipython-input-7-8fa859f51091>:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'Article ID': article_id, 'LLM Summary': summary}, ignore_index=True)
<ipython-input-7-8fa859f51091>:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = r

In [ ]:
results_df

NameError: ignored

In [ ]:
df.loc[2, 'Original Article']

"-lrb- cnn -rrb- governments around the world are using the threat of terrorism -- real or perceived -- to advance executions , amnesty international alleges in its annual report on the death penalty . `` the dark trend of governments using the death penalty in a futile attempt to tackle real or imaginary threats to state security and public safety was stark last year , '' said salil shetty , amnesty 's secretary general in a release . `` it is shameful that so many states around the world are essentially playing with people 's lives -- putting people to death for ` terrorism ' or to quell internal instability on the ill-conceived premise of deterrence . '' the report , `` death sentences and executions 2014 , '' cites the example of pakistan lifting a six-year moratorium on the execution of civilians following the horrific attack on a school in peshawar in december . china is also mentioned , as having used the death penalty as a tool in its `` strike hard '' campaign against terroris

In [ ]:
# # Assuming results is a dictionary or some data structure
# from pprint import pprint

# pprint(results)  # Pretty print the results

# # Save the results to a file
# with open('results.txt', 'w') as f:
#     f.write(str(results))

# # Download the file
# from google.colab import files

# files.download('results.txt')


In [ ]:
hyperparameters = {
    "temperature": 0.2,
    "max_output_tokens": 1024,
    "top_p": 0.8,
    "top_k": 40
}

prompt_text = "Give me a full summary for this: "
# Concatenate the prompt with the article content
formatted = prompt_text + a

generated_summary=llm.predict(formatted, **hyperparameters)
generated_summary

''

In [ ]:
!pip install rouge
!pip install pyrouge

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyrouge: filename=pyrouge-0.1.3-py3-none-any.whl size=191604 sha256=1400bbb28589c3dd92e2fa752457ed4747390fa35dfb2f7f7dcc4cc55d89a358
  Stored in directory: /root/.cache/pip/wheels/9a/67/12/c5dd8ef8b4152bb8789eafd2a74a734e2dc7bb9eae02b768e7
Successfully built pyrouge


In [ ]:
from rouge import Rouge

# Initialize Rouge
rouge = Rouge()


# Create a new DataFrame to store the results
df_results = pd.DataFrame(columns=['Article ID', 'Original Article', 'Highlight', 'Generated Summary', 'R1-R', 'R1-P', 'R1-F', 'R2-R', 'R2-P', 'R2-F', 'RL-R', 'RL-P', 'RL-F'])

# Iterate over the rows of the original DataFrame
for index, row in df.iterrows():
    # Generate summary
    article = row['Original Article']
    generated_summary = llm_chain.run(article)  # Replace this with your actual summary generation code
    # Calculate ROUGE scores
    reference_summary = row['Highlight']
    scores = rouge.get_scores(generated_summary, reference_summary)[0]  # Get the first (and only) score

    # # Append the results to the new DataFrame
    df_results = df_results.append({
        'Article ID': row['Article ID'],
        'Original Article': article,
        'Highlight': reference_summary,
        'Generated Summary': generated_summary,
        'R1-R': scores['rouge-1']['r'],
        'R1-P': scores['rouge-1']['p'],
        'R1-F': scores['rouge-1']['f'],
        'R2-R': scores['rouge-2']['r'],
        'R2-P': scores['rouge-2']['p'],
        'R2-F': scores['rouge-2']['f'],
        'RL-R': scores['rouge-l']['r'],
        'RL-P': scores['rouge-l']['p'],
        'RL-F': scores['rouge-l']['f'],
    }, ignore_index=True)

In [ ]:
llmcsv=pd.read_csv("/content/llm_with_roguescores.csv")

In [ ]:
llmcsv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 771 entries, 0 to 770
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Article ID   771 non-null    object 
 1   LLM Summary  771 non-null    object 
 2   llmR1-R      771 non-null    float64
 3   llmR1-P      771 non-null    float64
 4   llmR1-F      771 non-null    float64
 5   llmR2-R      771 non-null    float64
 6   llmR2-P      771 non-null    float64
 7   llmR2-F      771 non-null    float64
 8   llmRL-R      771 non-null    float64
 9   llmRL-P      771 non-null    float64
 10  llmRL-F      771 non-null    float64
dtypes: float64(9), object(2)
memory usage: 66.4+ KB


In [ ]:
P=['0408464fb6eb2702626d2ca1cbb7c28e76ddd255', '18f7505fa80eaa981a2f52fbb96fcea5908cd4ee', '97163bb36cb49730a634a0cbaa01108c2fd89e01', 'cd421645a56af17128c17a41ec2c9074bce84ddc', '4fde6cc846d0709e25a9cccec8d197ba26c15c82', '5f5a8814d8ec3c16299782db7c724cb771ea5adb', '56516cebdf30bd22dc8353549b70b4bb33993753', '446fdf91f59be259167ae59b0a3de1a664891751', '3d58a02a6578e08c2bf7494263ef9a083620e1f6', 'c077e15f710484d2cb3ffe26cd9b331353cb74e2', 'fd7a007f17a13ee329cba78b298bdd77d4d33d42', 'ba2ec653b7d0c46759f39aef6dac850c9229716f', '4cfe02fe65eedf9364dd2b9633dca27349ec02aa', '58b0d34958a722a453fe24deed240e08e7be8ced', '6abc848961744dde2da8a4b8e5337f34d32d45ed', '62545d81f24bc768afd19875d9e62e38a6ddb97d', 'efdb6043cf2ac2663137f8cb8696f0735ee58b86', 'bde8e22f3e1c3af2ca0f09f734f33d67140ef1aa', '9eccb748cdaefd4c472101db76f347914f850c1a', '685b0b902225d5b807a721e294f5ae16febf1f0e', 'b4c8c6a6ad6257b5d8dfd82b80cd3b0fbdd342e3', 'e79e40746dbc24643a3a3f044d2f49689ff82e38', '539d895fd424cbda3e4985c6aee36e272401b359', 'db61d01f82dc63596f7e2fa93b4ab4ae278daf1f', 'c331159c09eee2e7a27b90d973500e7b3db04d94', '1f4aaf00cad1c6272240bd109fd34201b92a0937', '436772093dd75272d1d867c80e6b142329fcfa3e', '6de323acb06b2590f5aba38ae450865ea4e793b1', 'ebccdbf5779c2d3fa7180747e333dde756aa62fb', '2bc2c4566f364d664331809744ff9a40841d9617', '975132a0649fbebbdb6cdd32534b165eece0053a', '3612c620d3fa1eafc99c0c154ad5caf7481727f4', '31edccbcedf3d7e2819818f109a35ecdfcd65970', 'b8d8f6a6518138c8b1e0e845f46e62e2b7fe3fd8', 'de45d9d92d7ee486a71ba13f4cc52470b9df5c01', 'be8918aac4ce45cfefddaa245c26d9314d1194d4', '3415b0e398cd14d27f386b28a61f4d585b8eaa11', '7623d447bab49c8e99dfc99289b2281a1fed8d4f', '45071cfaf26fa14f1a8a9276393401f54e0f93ca', '837970f6b544e927c6fe2261a81f74f8313f2bdf', '47609113b2709b46ccafb3782980cd3da7ab7d2d', 'c446f86eea5f7d48f5cae27df669b80aff8de043', 'a84811a80358e86f9a62669fa69521a0ac12130a', 'bbdc0e58a203b016166f896936d9862ae9fd11be', '6c5ef5f577b97c98b8580476690883176584569e', '6a778af88ba885ef38fd88987afa1b6167e5c494', '2094d594b67b61d842fdc5ef4891b721e36171e7', '6af327c6e93d12422c1b605ff7581a813cdb1691', 'c94b92e2e7f31e2ed580137a02a5c60fa6537458', '540fc636957b4d049d6b4f07ae72c1b3be6ec020', 'c8fcf4c23a15895a2c7272ec04c5a92b06575d5e', '3a407cf2ec50f113c6300b7a39936c229d1b29f4', '1d17d402757c7da68c499b95368f477ad5d66513', 'a382fc98c0652dea0c2275a4110b744476b30ea3', '949378fdf5114619911a5e0fe0182d45f1c27c17', '9dad1056763b4580aed22d85670a801c8c3f352c', '644da4a1c3baef155cf8c4675008540a8b777988', '35d5a717be7c0144d24c34233e5321e871dc614e', '8000bfa6f05e428b2c7cc552d0df4c36283108fd', '7be8bc293e448903d2b23f4cea091b0407848897', 'bfcee0d2eed31196bb00e1f52606a3f168eee17c', '2b4eb5a283f51b54e1b0318936ae9bcfa70ce243', 'd75247e02a692fc343bc9826662870011eaed521', '4b637c62b6ac747871a3051a97f29dd282633360', '270fdae145aa5ae3ae73adcf44be72cec339dbbe', '60c3114cd31fced2982a5e912f2224220d6c5254', '32259d1946a7e376f3915aea7f7b4a4e14cbe02f', '80a28afe6ded1ad0ddd6f2ba0ac9b66436e91739', 'c3881866bcacbb55bd959b15a197bae3eb8b3384', 'f5ff04170c39e33afdf58bb732dd03265662c4c0', 'cb7a72d78d5f34b9300f8486e40e57ffcf9d98d5', 'f1beee2435f59b05e0da484c7a1c167709412d6d', 'e1502ae8cb04cbdceee368551cfd65bcf6a16f07', 'e6e0a4b568f6214c18c38cfcac5a10ecfb0dbf6e', '623ad83ebb32752afaa1fb4b134ab697eae8b798', '0e6dd3f40daaff5c14f0a34ba75e9f05c40a55c8', '0d1d6284061a99cb2478ecb4b490e4605698ec3e', 'de24115848174d0534512cd251b1d53cd343f23e', '574767c29dfee14ea227639f45387c07437a55e5', 'a8a2ace82e62b4020b5a241060d4a548f7ee3a7f', 'bef9d7ac62dac0fb9a200d2acae6d333388ec7f1', '6426eaaa8fe11407ed2290d9c0ad2bc8e1a723d5', 'bb8a7a6bc30f5d10bfd747daa4887134a780d66d', 'f7dd3331408529408be2b706392e0b4a02a6a4cc', '84a933b7edc70c7f5e6ed6558c7b57c100abc114', '9f4ab634ce7925b5527345ba0efc583b206c47a2', 'f800996f2cca42b205a7cfd37805fb77d577254b', 'd6f928d950f7d97c9fc3a5f575b9eaa325a7dd4d', '8f324e3064f6cdc9c378dfcf4545ed2c124e7103', '32069f98ff75ce1a65beeffcf84cd80f92bae5dc', '7f13d9dff7707609a042a5c3e99671c98de8c135', '4dcf206f63daa3ffdb4c77b2441e7aed78d2c8c5', '864240ad271c7da6c74823abf2abfb2b98f427cb', '6299e17b582fffbd6732d237dcc06c97c5d935ff', 'c3cb61c858ec4a178760b619430ddff7e03eb0c8', '0a8fd2888f20c9e705696d8ef847a32df47900e6', 'b68be4e8fe382de849954933357e4a32409c20ff', '34f1b80e4e3fbd60dd942b3e7e52a7f8c61f06ef', '52343da9a3ef719280b4b6946e6c500e819552d0', '2f6106809de41d59f422d527900a6b344174c5b2', '69406e34836ea1f7680d4298e83fd6d83a6c7997', 'ef2d42928cc3b1762d0bbb6037db308db53fc864', '31809d739569d22b663fc1cdbfb1fb4206ade734', '98d980ba9bc5479f9c340a18a61941d5ccb52774', 'dbba46c94632ad510072e1b52c4f427c4dadca60', '65676acfb425e2a29cd7fca216174635ea408382', '16b9db5d22a87f739c918886c6b3d7acdfcfd58a', '30f2056f43c0a58a261c4feddefd131d498199cb', '5231a93ff48e3dcdc22ea9df981237c26a474714', '29503dc74ccb039c4c1562844299116a545dea39', 'd34233997a6d71174a09a8d8001ec64468a2f640', 'a37aa9e7b6985f710c3ad79cd0b53619b5e0bf5a', '707c09009396d151d2d4f819925f78e446151f39', '33ca9fba99197c82369e99aa3d234783e719227c', 'faf1d451489b43676734ea706926885bbbedfce3', '0c5ec67f7b1d7a2fdb0a3f3ffb8263d8d9ddf872', 'd9e8e6f9c0255bf26881a2eabba8f5cbe733c55f', '9625484feb731f45fd8059112238d94f141d413c', 'ca3b2dc93f170f06e3a60fe7cd3243d204cc4b75', '2103c883437dbd9f0c17885f60190dabb4ab56dc', '30944082061b9e844391dcbc98775f22cb3ef791', '202c35ec48be21fc196139dd538db271c0f65e4a', '280d5930fb82ad2958fd1dcf452c6a4f1b5ad39f', '71efc54d9a237c40c469965cc24841f9f9a7b019', '3291399360e20650ea565c27371a72c7b448082e', 'e67604ecb75aa21391e1d15a577d16b8db990de5', 'e2a44a3e3edd44c394ad6ff7207980f7f1e9b4b3', '7661803c32a1511bf302a8a5abdc3c462f080fb2', 'de84417cc1ffcb6ecabfab7600c232eb9fd11915', '8a7466a5c8516b2087aea24647cc07b508697ab6', '20a6b5894c4cd75e32f6f4946467854aeb539337', '7113eca13e8f3dd7760e609fdc97660de1806d2a', '61254109741e1db17ba31582d1c783d282e6bbf6', '78b863c5bce9b8fc0d743bc396b0308a9eb17a77', '92151f31c3478c5e0e4352a0f7323c3acfc297e7', 'cabca761a3814275d3d1b2b6c7cfc1f9cd3810b7', '878c4af5ebfb7a1831af90ad0d6e7bdcc4a3d4b8', '1dd00c89d71a5611797bb34da8a6eada8f058405', '711e00313310a956641a47791fdf8299ee06b2c2', '01b0585316ca58505df1d4ace2d888d0e50e7600', 'c11e656ad31a4f14b0d276290a757a0da6a73ba4', '4b132f90a0f4369df9b7ecba759c5aa068dd12be', 'd3e9f0aca4e608fc2a3b85115b96cb0c453191af', 'c18fe1d648b40aa763644c37a9a2a562a340e172', '9ad4662fc2d978672ae4b5eea4e4806ae6a7d076', 'db9f358d0e08da7cd0276453f8bcdc114ff06fdb', 'b19ab7fa421bcbb921c5a20718db5918091bb15d', '9268b9d826db3d8acf31a03bdc3e30cd75cc93de', 'be5d8524bf21d485e203425d764ed60a3bf71b8d', 'f97b9b19e5a90dce0fc0b098f2a7f70ec36445fb', 'e84c74b45a589a6faefaa881016505fd282a46ca', 'd495bb584d5fda76f9b90beca528469f5bcfc5b3', 'f7f9156f26bbf12e63523bdd2f607a2f7a7f9f0c', '7e91926e47bdaa4054e3a69784bee8ee27d0dd4b', '2434b0ec4419fd48c0d15e543030555129398ec6', 'f0dba7fa5d45ffb29070afda0c4b230c8f318a51', '82461522ee90f8f5be2ec226e2016ad40588866d', 'a3830cf0f4311a3f816a710d61d7e14ac9ffa9fe', 'bc9d5ab2c343edd6522ee4ebe63c5ca4adcebb1b', '9638b449931f0f4a21167997d37d5dd43ee2430f', 'e8b93366b9c9c5bf4409dc1f8064122063ccd4ca', '3a8e2afdba04d7356526c94232f17eef9a5e4ddd', '7ae16f8ac4a86a6de4669e130e068d9f7e492ddf', '1b3bdc7fa131c4c82e8e996c936abe0da840e2ef', '5d1577700e760d853376d15ad75d8972119d6ddb', 'd6bbbfa9b5c58e99d21d549a9fb5b512b342e301', 'e0a415bf04e1929507bb8626cdf6379169264838', '5c3e16c415b463269527cafd220170e1303048f8', '09085a0b380bcbc205c7a3902f4d5492da2eacda', '223ecfeb056056991eb877fbd63ce66ca616d372', 'f708809379316d68524802c6ca052985bbe248cd', 'c38207a8ba7332bf8fc449770b2554510fa1482a', '3bdec3398b4dd68453d97ab7718086655843e899', 'a7827e12988bd89e4f1079705079a135beefc1f2', '34c33427589efd00f343e49e2e5109d444d560f5', '198f9a4a72b169e04ec003cb4717f46dc0d3191f', '9ef9bcf7b102537cff2774dd7be2c6eefd5b80fd', '7e4d197d5fe8272c0471199cf9987f3786a80c2b', '1fda6d5647b826db2ab920c945dd7944dbdfbde9', '2805d03b2ec2d8a1e5343f71d70912db2a58736e', 'ddcbe22893aa887ff799cf3e40bbe2a536965f91', 'b6d1933d8d4a8c1b6b76d44ec3ed69d86268a7b3', 'ad58f533f9eaf52080929e32c5dd5eaf2bafba0e', 'f2a9f3c80876424da8cee335cb0e0467caa5f3ca', '5ba4a7a9733a497625971981764184cb75618b81', 'a659d867e198efa85e02b90d5e78d2f6a6e60fb7', '36c34f4ffa8815711beea7d2fda86eea047b8802', 'a4b00d5247eef69b5d51cab1f1883da314211366', 'cca91bf7af4db82293421776f5299457dce34909', 'e248a8e2416dddfa8899f0781b967429232063b3', 'e266b3daac6d9e72e637e8f83db97621c5beafd0', '9f2dbe8953dd5b1d2b0f2f748c51bfccbe72906e', 'a2b66e4376258cd41604d654a395f7540d2e8514', '2edc424a6d80e2e10a46b1e8b52f14c481cc1daa', '0cdf69eb6b6860142724e940cf9f75d93907227f', '34ad340fa41e205b95b37b00e8ccbc38afb3e68f', '117444537cbffcb807dc04565f39dca46f7d2cef', 'ecb7f439cc992286b8933b117bb6c498bd133bcd', 'b6ee1b3428dda9063461deefc0086c2d66279d0f', '42d88bafeb5f838a0be02c3eb97fe12f5df3d92f', '4c976ed4cb92068d476b1c17753715948e39c62f', 'a5fffe339cda4c8e002d992ab93888559107f3ab', 'd6fc70d27119a3705b96ace6b20b9fd2f3cfcbca', '89fed7685538cd358e611932653eea931d7b057f', '5effc218f677590dbc9037e196549b5341242933', 'b8a47e9569e9d0919638f5bed6f13e565f84e855', '35c590581dcd2a417101f2ef80f63f69cdfe4030', 'f99b61fb576e893ffbd2e6d6f494b9dd0fbf8941', '5dd71e881a79e27363e2f73c3afa80b8cc3e5a64', '64b3b703816b40851bd4b7c0255df5834e136611', 'cf5e6722047569c81b635be96ef0b07cb04eed73', 'ffd4fd3ed5e3d498d6135fc93ae6c9170d10611c', '159ab1d0f729f77e86a6a77bf43d0291d069c66c', 'e0a3acedc34d92569ce5f6032d504c35fbf30720', '0e5130d172cdaaa2afcb51585f4aa0580b049431', 'a776fa2caa0fa9cac0ce5bf71d2bf44eb7a7346a', '1027d076dbb423b90447f6f3f8315610aefd682c', '0322c7a3366d45134e8ba78c7f369d2f2cd14176', '0db1a9abd0524d51270531e3a80d36afaa7550fe', '0d29ae06dd015e141646c3b03671901c1b8465c4', 'e2ef75518e69947f79acb9202516689b6c869e92', '16bf321c82f6ffe7f20479a12a9e432685f2d91b', 'c37431a19d8433f20447858d1c810b7b77804473', '77685c1f281a3e7472b51f6619d7d54c6bc0c717', '6d76062fb07e8a37609bbd527934a9219929560b', '083b6c3239f969ae12491f14fc09b1f17d396983', '1afd2df29d4e642f20130a231d27004216667fe1', '8749719d4fb9a14d833d39cde0b6e07985bee823', '43959f19ee64571fead0661415d4e13c54d71ee1']